In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import mysql.connector    # installing pymysql and mysql-connector package for making connections to database server
!pip install pymysql  
!pip install mysql-connector
pd.set_option('display.max_columns',None)
import warnings
warnings.filterwarnings('ignore')
cursor=connection.cursor()
cursor.execute('show databases')
for i in cursor:
    print(i)
project= pd.read_sql_query("show tables",connection)
print(project)
data = pd.read_sql_query("select * from dataset_list",connection)
data
data.info()
data.head()
data.tail()
data.shape
data.describe()
data.columns
data.select_dtypes(include=['int64','float64']).columns
data.select_dtypes(include=['object']).columns
import sweetviz as sv # library for univariant analysis
my_report=sv.analyze(data) # passing the original dataframe
my_report.show_html() # arguments will generate to// the library
data.select_dtypes(include=['object','category']).columns.tolist()
for i in data.columns:
    data[i].replace("", np.nan,inplace=True)
data.isna().sum() #cheking the Nan values
# Dropping Column which have high null values.
data.drop(columns=['Reopen_Time'], inplace=True)
data.drop(columns=['No_of_Related_Incidents'], inplace=True)
data.drop(columns=['No_of_Related_Changes'], inplace=True)
data.drop(columns=['Related_Change'], inplace=True)
# lets fill all the null values with statistical methods
data.loc[data["CI_Cat"].isnull(),"CI_Cat"] = 'application'
data.loc[data["CI_Subcat"].isnull(),"CI_Subcat"] = "Server Based Application"
data.loc[data["No_of_Reassignments"].isnull(),"No_of_Reassignments"] =0
data.loc[data["Handle_Time_hrs"].isnull(),"Handle_Time_hrs"] = 0
data.loc[data["Closure_Code"].isnull(),"Closure_Code"] = "Other"
data.loc[data["No_of_Related_Interactions"].isnull(),"No_of_Related_Interactions"] = 1
data.isnull().sum()
data.duplicated().sum() #checking duplicates
from sklearn.preprocessing import LabelEncoder
enc=LabelEncoder()
#CI_Name
data.CI_Name =enc.fit_transform(data.CI_Name )

#CI_Cat
data.CI_Cat =enc.fit_transform(data.CI_Cat )

#CI_Subc
data.CI_Subcat =enc.fit_transform(data.CI_Subcat )

#WBS
data.WBS =enc.fit_transform(data.WBS )

#Incident_ID
data.Incident_ID =enc.fit_transform(data.Incident_ID )

#Status
data.Status =enc.fit_transform(data.Status )

#Impact
data.Impact =enc.fit_transform(data.Impact )

#Urgency
data.Urgency =enc.fit_transform(data.Urgency )

#Priority
data.Priority =enc.fit_transform(data.Priority )

#number_cnt
data.number_cnt =enc.fit_transform(data.number_cnt )

#Category
data.Category =enc.fit_transform(data.Category )

#number_cnt
data.number_cnt =enc.fit_transform(data.number_cnt )

#Category
data.Category =enc.fit_transform(data.Category )

#KB_number
data.KB_number =enc.fit_transform(data.KB_number )

#Alert_Status
data.Alert_Status =enc.fit_transform(data.Alert_Status )


#Open_Time
data.Open_Time =enc.fit_transform(data.Open_Time )


#Resolved_Time
data.Resolved_Time =enc.fit_transform(data.Resolved_Time )

#Close_Time
data.Close_Time =enc.fit_transform(data.Close_Time )



#Closure_Code
data.Closure_Code =enc.fit_transform(data.Closure_Code )


#Related_Interaction
data.Related_Interaction =enc.fit_transform(data.Related_Interaction )

data['No_of_Reassignments'] = data['No_of_Reassignments'].astype(int)
data['Handle_Time_hrs'] = data['Handle_Time_hrs'].astype(str)
data['No_of_Related_Interactions'] = data['No_of_Related_Interactions'].astype(int)
#No_of_Reassignments
data.No_of_Reassignments =enc.fit_transform(data.No_of_Reassignments )

#Handle_Time_hrs
data.Handle_Time_hrs =enc.fit_transform(data.Handle_Time_hrs )


#No_of_Related_Interactions
data.No_of_Related_Interactions =enc.fit_transform(data.No_of_Related_Interactions )

data.info()

corr_data=data[['CI_Cat', 'CI_Subcat', 'WBS', 'Incident_ID', 'Status', 'Impact',
       'Urgency', 'Priority', 'number_cnt', 'Category', 'KB_number',
       'No_of_Reassignments', 'Open_Time', 'Resolved_Time', 'Close_Time',
       'Handle_Time_hrs', 'Closure_Code', 'No_of_Related_Interactions',
       'Related_Interaction']]
plt.figure(figsize=(23,25))
sns.heatmap(corr_data.corr(),annot=True)

plt.figure(figsize=(14, 14))
plotnumber = 1

for i in data:
    plt.subplot(6, 6, plotnumber)
    sns.boxplot(x=data[i])
    plotnumber += 1
    if plotnumber > 27:
        break

plt.tight_layout()
plt.show()

X=data.drop('Priority',axis=1)
y=data.Priority

X.shape

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape)
print(X_test.shape)

X_train
X_test
y_train
y_test

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Let's see oue target variable is balanced or not if it's not balanced. So, we have to apply SMOTE for balanced the data.
y.value_counts()
# here we can see our target variable is not balanced.

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Creating a linear regression model
model = LinearRegression()

# Training the model on the training data
model.fit(X_train, y_train)

# Making predictions on the testing data
predictions = model.predict(X_test)

# Calculating and printing mean squared error (MSE)
mse = mean_squared_error(y_test, predictions)
print("Mean Squared Error:", mse)

# Calculating R-squared
r2 = r2_score(y_test, predictions)
print("R-squared:", r2)

from sklearn.tree import DecisionTreeRegressor  # Importing DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score  # Importing regression metrics

# Creating a DecisionTreeRegressor object with specified hyperparameters
dt = DecisionTreeRegressor(max_depth=10, min_samples_split=30, min_samples_leaf=20, random_state=42)

# Training the model
dt.fit(X_train, y_train)

# Predicting on the test set
y_hat = dt.predict(X_test)

# Evaluating the model using Mean Squared Error (MSE) and R-squared
mse = mean_squared_error(y_test, y_hat)
r2 = r2_score(y_test, y_hat)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

# Training score
y_train_predict = dt.predict(X_train)
mse_train = mean_squared_error(y_train, y_train_predict)
r2_train = r2_score(y_train, y_train_predict)

print("Mean Squared Error (Training):", mse_train)
print("R-squared (Training):", r2_train)

# Predicting on the test set
y_hat = dt.predict(X_test)

# Evaluating the model using Mean Squared Error (MSE) and R-squared
mse = mean_squared_error(y_test, y_hat)
r2 = r2_score(y_test, y_hat)

print("Mean Squared Error (Test):", mse)
print("R-squared (Test):", r2)

from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score

# Create a DecisionTreeRegressor object
dt = DecisionTreeRegressor(random_state=42)

# Define the grid of hyperparameters to search
param_grid = {
    'max_depth': [3, 5, 7, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=dt, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')

# Perform hyperparameter tuning on the training data
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Use the best model for predictions
best_model = grid_search.best_estimator_
y_hat = best_model.predict(X_test)

# Evaluate the model using Mean Squared Error (MSE) and R-squared
mse = mean_squared_error(y_test, y_hat)
r2 = r2_score(y_test, y_hat)
print("Mean Squared Error (Test):", mse)
print("R-squared (Test):", r2)
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Creating a RandomForestRegressor object with 100 decision trees
rf_reg = RandomForestRegressor(n_estimators=100, random_state=42)

# Training the model
rf_reg.fit(X_train, y_train)

# Predicting on the test set
y_pred = rf_reg.predict(X_test)

# Calculating the F1-score (this will result in an error since F1-score is for classification)
# f_score = f1_score(y_test, y_pred)

# Instead, let's evaluate the model using Mean Squared Error (MSE) and R-squared
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import RandomizedSearchCV

# Define the parameter grid
n_estimators = [int(x) for x in np.linspace(start=200, stop=2000, num=5)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num=5)]
max_depth.append(None)
min_samples_split = [30, 25, 10]
min_samples_leaf = [5, 10, 15]
bootstrap = [True]

# Create the random grid
random_grid = {
    'n_estimators': n_estimators,
    'max_features': max_features,
    'max_depth': max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf,
    'bootstrap': bootstrap
}

# Create the base model
rf_reg1 = RandomForestRegressor(random_state=42)

# Initialize RandomizedSearchCV
rf_cv = RandomizedSearchCV(
    estimator=rf_reg1,
    scoring={'mse': 'neg_mean_squared_error', 'r2': 'r2'},  # Use MSE and R2 as scoring metrics
    refit='mse',  # Choose one of the scoring metrics for refitting
    param_distributions=random_grid,
    n_iter=100,
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

# Fit RandomizedSearchCV to the data
rf_cv.fit(X_train, y_train)

# Get the best parameters
rf_best_params = rf_cv.best_params_
print(f"Best parameters: {rf_best_params}")

# Create the RandomForestRegressor with the best parameters
rf_reg2 = RandomForestRegressor(**rf_best_params)

# Train the model with the best parameters
rf_reg2.fit(X_train, y_train)

# Make predictions
y_predict = rf_reg2.predict(X_test)

# Calculate the Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_predict)
print("Mean Squared Error:", mse)

# Calculate the R-squared (R2)
r2 = r2_score(y_test, y_predict)
print("R-squared:", r2)
from sklearn.ensemble import GradientBoostingRegressor  # Import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score  # Import mean_squared_error and r2_score for regression

# Model creation
gbm = GradientBoostingRegressor()  # Object creation

# Fit the model
gbm.fit(X_train, y_train)  # Fit the data

# Predict on the test set
y_gbm = gbm.predict(X_test)  # Predict prices

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_gbm)  # Calculate Mean Squared Error

# Calculate R-squared
r2 = r2_score(y_test, y_gbm)  # Calculate R-squared

mse, r2  # Print Mean Squared Error and R-squared
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score


# Creating a Gradient Boosting Regressor
gbm = GradientBoostingRegressor()

# Define the grid of hyperparameters to search
param_grid = {
    'n_estimators': [50, 100, 150],  # Number of boosting stages to be run
    'learning_rate': [0.05, 0.1, 0.2],  # Learning rate shrinks the contribution of each tree
    'max_depth': [3, 4, 5]  # Maximum depth of the individual estimators
}

# Perform grid search with cross-validation
grid_search = GridSearchCV(estimator=gbm, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Use the best model for predictions
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Calculate mean squared error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

# Calculate R-squared
r2 = r2_score(y_test, y_pred)
print("R-squared:", r2)
from xgboost import XGBRegressor  # Import XGBRegressor for regression
from sklearn.metrics import mean_squared_error, r2_score  # Import mean_squared_error and r2_score for evaluation

# Model creation
xgb_r = XGBRegressor()  # Object creation

# Fit the model
xgb_r.fit(X_train, y_train)  # Fit the data

# Predict on the test set
y_hat = xgb_r.predict(X_test)  # Predict prices

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_hat)  # Calculate MSE

# Calculate R-squared
r2 = r2_score(y_test, y_hat)  # Calculate R-squared

mse, r2  # Print MSE and R-squared
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score

param_grid = {'gamma': [0,0.1,0.2,0.4,0.8,1.6,3.2,6.4,12.8,25.6,51.2,102.4, 200],
              'learning_rate': [0.01, 0.03, 0.06, 0.1, 0.15, 0.2, 0.25, 0.300000012, 0.4, 0.5, 0.6, 0.7],
              'max_depth': [5,6,7,8,9,10,11,12,13,14],
              'n_estimators': [50,65,80,100,115,130,150],
              'reg_alpha': [0,0.1,0.2,0.4,0.8,1.6,3.2,6.4,12.8,25.6,51.2,102.4,200],
              'reg_lambda': [0,0.1,0.2,0.4,0.8,1.6,3.2,6.4,12.8,25.6,51.2,102.4,200]}

# Initialize XGBRegressor
XGB = XGBRegressor(random_state=42, verbosity=0, silent=0)

# RandomizedSearchCV
rcv = RandomizedSearchCV(estimator=XGB, scoring='neg_mean_squared_error', param_distributions=param_grid, 
                         n_iter=100, cv=3, verbose=2, random_state=42, n_jobs=-1)
                        
rcv.fit(X_train, y_train)  # Training data on RandomizedSearchCV

cv_best_params = rcv.best_params_  # Get best parameters
print(f"Best parameters: {cv_best_params}")

# Initialize XGBRegressor with best parameters
XGB2 = XGBRegressor(**cv_best_params)
XGB2.fit(X_train, y_train)  # Training

y_predict = XGB2.predict(X_test)  # Testing

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_predict)

# Calculate R-squared
r2 = r2_score(y_test, y_predict)

mse, r2  # Print MSE and R-squared
from sklearn.neural_network import MLPRegressor

# Create MLPRegressor model
model = MLPRegressor(hidden_layer_sizes=(50, 3),
                     learning_rate_init=0.1,
                     max_iter=100,
                     random_state=10)

# Train the model
model.fit(X_train, y_train)

# Predict target values for the test set
y_predict = model.predict(X_test)

# Predict target values for the training set (optional, for evaluation)
y_train_predict = model.predict(X_train)


# Evaluate the model
mse_train = mean_squared_error(y_train, y_train_predict)
mse_test = mean_squared_error(y_test, y_predict)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

# Calculating R-squared
r2 = r2_score(y_test, predictions)
print("R-squared:", r2)
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score

# X = Your feature matrix
# y = Your target vector

# Initialize SVM regressor
svm_regressor = SVR(kernel='linear', C=1.0)

# Train the regressor on the training data
svm_regressor.fit(X_train, y_train)

# Predict target values for the test set
y_pred = svm_regressor.predict(X_test)

# Calculate mean squared error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

# Calculate R-squared
r2 = r2_score(y_test, y_pred)
print("R-squared:", r2)